In [2]:
#!/usr/bin/env python
import snowflake.connector
import pandas as pd
import numpy as np
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
###
# https://docs.snowflake.com/en/user-guide/python-connector-example.html
from glob import glob

1) Reading RAW data and data Cleaning and Transformation Process 

In [46]:
filenames = glob('raw_datasets/*/20*.csv',recursive=True) 
print(len(filenames))
count=0
for file in filenames:
    # Read the data files individually 
    data = pd.read_csv(file)
    print('Reading file ==>' ,file)
    #Cleaning the data base for null values and replacing the with corrresponing valuse and droping unwanted columns
    data=ET_load(data)
    print('Cleaned file ==>',file)
    #Saving the cleaned file to to new directory
    p = 'processed_datasets/'+file[21:] 
    data.to_csv(p,header=None,index=False)
    print('Saving File ==>',file)
    count=count+1
    print(count,'/',len(filenames),'Processed!')


534
Reading file ==> raw_datasets/2019-03/2019-03-hampshire-street.csv
~~Empty values replaced!!
~~Dataset Cleaned
~~Duplicated Entries Removed
Cleaned file ==> raw_datasets/2019-03/2019-03-hampshire-street.csv
Saving File ==> raw_datasets/2019-03/2019-03-hampshire-street.csv
##########################################
Reading file ==> raw_datasets/2019-03/2019-03-south-yorkshire-street.csv
~~Empty values replaced!!
~~Dataset Cleaned
~~Duplicated Entries Removed
Cleaned file ==> raw_datasets/2019-03/2019-03-south-yorkshire-street.csv
Saving File ==> raw_datasets/2019-03/2019-03-south-yorkshire-street.csv
##########################################
Reading file ==> raw_datasets/2019-03/2019-03-essex-street.csv
~~Empty values replaced!!
~~Dataset Cleaned
~~Duplicated Entries Removed
Cleaned file ==> raw_datasets/2019-03/2019-03-essex-street.csv
Saving File ==> raw_datasets/2019-03/2019-03-essex-street.csv
##########################################
Reading file ==> raw_datasets/2019-03/2019

**Counting number of output files 

534


2) Connecting to Snowflake 

In [3]:
# Create main connection context
con = snowflake.connector.connect(
    user='anuragvuppala',
    password='Snow@Flake44',
    account='vx24548.europe-west2.gcp'
)

Connection Validification

In [4]:
c = con.cursor()
c.execute("SELECT current_version()").fetchone()
c.execute("USE ROLE ACCOUNTADMIN") 

Database and table creation

In [5]:
c.execute("CREATE WAREHOUSE IF NOT EXISTS STREET_CRIME_WH")
c.execute("CREATE DATABASE IF NOT EXISTS STREET_CRIME_DB")
c.execute("USE WAREHOUSE STREET_CRIME_WH")
c.execute('USE DATABASE STREET_CRIME_DB')
c.execute('USE SCHEMA PUBLIC')

In [63]:
c.execute('CREATE TABLE IF NOT EXISTS "CRIME_DATA_TABLE" (crime_id STRING, month STRING, reported_by STRING, fall_within STRING, longitude FLOAT, latitude FLOAT, location STRING, lsoa_code STRING, lsoa_name STRING, crime_type STRING, last_outcome STRING, id integer not null constraint unique_id primary key not enforced)')

In [66]:

c.execute("PUT file://processed_datasets/20*.csv @%crime_data_table")



In [67]:
c.execute("COPY INTO CRIME_DATA_TABLE")
c.execute("Remove @%crime_data_table")
print('Romoved files form Stageing area!')


Counting Total number of crime by crime type 

In [8]:
crime_count_type=c.execute("SELECT crime_type,COUNT(crime_type) FROM CRIME_DATA_TABLE GROUP BY crime_type ORDER BY COUNT(crime_type) DESC; ").fetchall()
crime_count_type

[('Violence and sexual offences', 5683339),
 ('Anti-social behaviour', 2887715),
 ('Criminal damage and arson', 1501087),
 ('Public order', 1364188),
 ('Other theft', 1243688),
 ('Vehicle crime', 1098493),
 ('Burglary', 893480),
 ('Shoplifting', 841126),
 ('Drugs', 543463),
 ('Other crime', 308124),
 ('Bicycle theft', 229767),
 ('Theft from the person', 227742),
 ('Robbery', 198963),
 ('Possession of weapons', 128387)]

Counting Total number of crime count for every street

In [18]:
crime_count_street = c.execute("SELECT month, fall_within, location, crime_type, COUNT(crime_type) FROM CRIME_DATA_TABLE GROUP BY location, month, crime_type, fall_within ORDER BY COUNT(crime_type) DESC; ").fetchall()
len(crime_count_street)
df=pd.DataFrame(crime_count_type)
df

ProgrammingError: 001003 (42000): SQL compilation error:
syntax error line 1 at position 53 unexpected '('.

In [71]:
con.close()

In [ ]:
df=pd.DataFrame(crime_count_type)
df